In [ ]:
import numpy as np
import msmrd2
import msmrd2.tools.quaternions as quats
import random
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyProteinMarkovSwitch
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS

In [ ]:
# Main parameters 
numparticles = 2
dt = 0.0001 #0.00001 #0.000005
bodytype = 'rigidbody'
particleTypes = [0, 1]
numBoundStates = 6
initialState = 1

# Parameters related to patchy Particle potential with angular dependence
# Define Patchy Protein potential parameters (This values are fixed and should match
# those used to determine metastable states in potential and trajectory.)
sigma = 1.0
strength = 60
angularStrength = 10
patchesCoordinates1 = [np.array([1.,0.,0.]), np.array([0.,1.,0.]), np.array([0.,0.,1.]),
                       np.array([-1.,0.,0.]), np.array([0.,-1.,0.]), np.array([0.,0.,-1.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
potentialPatchyProtein = patchyProteinMarkovSwitch(sigma, strength, angularStrength, 
                                                   patchesCoordinates1, patchesCoordinates2)

# Create unbound MSMlist (CTMSMs)
# MSM for particle type0
MSMtype = 0
seed = 0 # If seed = -1 uses random device as seed
markovModel0 = ctmsm(MSMtype, seed) # If no rate matrix specified, yields ratematrix [[0]]
D0list = np.array([1.0])
Drot0list = np.array([1.0])
markovModel0.setD(D0list)
markovModel0.setDrot(Drot0list)
# MSM for particle type1 
MSMtype = 1
ratematrix = np.array([[-0.5,0.5],[6.0,-6.0]])
seed = 0 # If seed = -1 uses random device as seed
markovModel1 = ctmsm(MSMtype, ratematrix, seed)
D1list = np.array([0.5, 1.0])
Drot1list = np.array([0.5, 1.0])
markovModel1.setD(D1list)
markovModel1.setDrot(Drot1list)
# Create MSMlist
MSMlist = [markovModel0, markovModel1] 

In [ ]:
# Set boundary
boxsize = 6
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, 'periodic')

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 0 # If seed = -1 uses random device as seed
bodytype = 'rigidbody' # Mix indicates there can be different types of rigidbody particles
integrator = odLangevinMS(MSMlist, dt, seed, bodytype) 
integrator.setBoundary(boxBoundary)
integrator.setPairPotential(potentialPatchyProtein)

In [ ]:
def generateParticleList(state, boxsize, types, unboundMSMs, randomSeed = -1):
    '''
    Generate a random particle list of two particles corresponding to an initial unbound state. As each
    state has several different configurations, this functions picks one randomly. The definition of the
    states is taken form patchyProtein.cpp (trajectory)
    :param state: bound state in which the two particle list should begin in (can be a list of states)
    :param boxsize: size of simulation box, if scalar it assumes the three box edges are the same in all dimensions
    :param types: array containing the particle types. The index should correspond to that of of the particle list.
    :param unboundMSMs: list of unboundMSM, needed to extract diffusion coefficients of particles.
    :param randomSeed: seed for python random generator. Important to specify in parallel runs. Default value of -1
    will use the default seed.
    :return: random particle list corresponding to one of the initialBoundstates.
    '''

    # Transform boxsize to vector if neccesarry.
    boxsize = boxsize - 1 # to ensure the whole compund is initially inside box (1 is the norm of relpos below)
    if np.isscalar(boxsize):
        boxsize = np.array([boxsize, boxsize, boxsize])

    # Transform state to list if neccesary
    if np.isscalar(state):
        state = [state]

    position1 = np.array([0.0, 0.0, 0.0])
    orientation1 = np.array([1.0, 0.0, 0.0, 0.0])
    # Define relative positions
    relPos = [None]*numBoundStates
    relPos[0] = np.array([1., 0., 0.])
    relPos[1] = np.array([0., 1., 0.])
    relPos[2] = np.array([0., 0., 1.])
    relPos[3] = np.array([-1., 0., 0.])
    relPos[4] = np.array([0., -1., 0.])
    relPos[5] = np.array([0., 0., -1.])
    # Define relative rotations
    rotations = [None]*numBoundStates
    rotations[0] = np.array([0.0, 0.0, np.pi])
    rotations[1] = np.array([0.0, 0.0, -np.pi / 2.0])
    rotations[2] = np.array([0.0, np.pi / 2.0, 0.0])
    rotations[3] = np.array([0.0, 0.0, 0.0])
    rotations[4] = np.array([0.0, 0.0, np.pi / 2.0])
    rotations[5] = np.array([0.0, -np.pi / 2.0, 0.0])
    # Convert axis-angle rotations to quaternions
    quatRotations = [None]*numBoundStates
    for i in range(numBoundStates):
        quatRotations[i] = quats.angle2quat(rotations[i])

    # Assign position and orientation depending on initial bound state (between 0 and 5)
    substate = np.random.choice(state) # if states was originally a scalar, always picks that value.
    position2 = relPos[substate - 1]
    orientation2 = quatRotations[substate - 1]
    p1State = 0
    p2State = 0
    # Obtain diffusion coefficients from unboundMSM
    D1 = unboundMSMs[types[0]].D[p1State]
    Drot1 = unboundMSMs[types[0]].Drot[p1State]
    D2 = unboundMSMs[types[1]].D[p2State]
    Drot2 = unboundMSMs[types[1]].Drot[p2State]
    # Define particles
    part1 = msmrd2.particle(0, p1State, D1, Drot1, position1, orientation1)
    part2 = msmrd2.particle(1, p2State, D2, Drot2, position2, orientation2)
    partlist = msmrd2.integrators.particleList([part1, part2])
    return partlist

In [ ]:
partlist = generateParticleList(initialState, boxsize, particleTypes, MSMlist, -1)

In [ ]:
# Check state calculated by trajectory matches with initial state chosen
part1, part2 = partlist[0], partlist[1]
dummyTraj = msmrd2.trajectories.patchyProtein(2,1)
boundState = dummyTraj.getState(partlist[0], partlist[1])
print("Initial condition bound state:", initialState)
print("Calculated bound state by trajectory:", boundState)
if (boundState != initialState):
    print("Warning, getState function (sampleDiscreteState) is not returning the correct state.")

In [ ]:
# Print particlelist into file and generates plotting tcl file for VMD
datafile  = open('../../data/vmd/testInitialCondition2.xyz', 'w')
# Other plotting parameters
farAwayPostion = np.array([30, 0, 0])
numSpheres2Draw= 15
# Define reference vectors for plotting small protein
refVec1 = np.array([0., 1., 0.])
refVec2 = np.array([-1., 0., 0.])
refVec3 = np.array([0., -1., 0.])
refVec4 = np.array([0., 0., 1.])
refVec5 = np.array([0., 0., -1.])
timesteps = 1 #100000
stride = 100 #10 #1000
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(numSpheres2Draw) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[2], part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[3], part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[4], part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[5], part.orientation)
                # Main particle 1
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                # Pacthes
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v6)) + '\n')
            if part.type == 1:
                v0 = part.position
                #if part.state == 0:
                v1 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(refVec1, part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(refVec2, part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(refVec3, part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(refVec4, part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(refVec5, part.orientation)
                # Main particle 2
                datafile.write('type_3' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_3' + ' ' + ' '.join(map(str, v6)) + '\n')
                if part.state == 1:
                    datafile.write('type_4' + ' ' + ' '.join(map(str, farAwayPostion)) + '\n')
                else:
                    datafile.write('type_4' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteinsMSV2(numparticles = numparticles, 
                                     outfname = "testInitialCondition2", 
                                     tclfname = "../../data/vmd/testInitialCondition2_2vmd.tcl")
print("Percentage complete: ", 100, " %")